## Vocabulary Analysis

In [1]:
%load_ext autoreload
%autoreload 2
import transformers
from transformers import AutoModel, AutoTokenizer

model_name = 'dccuchile/bert-base-spanish-wwm-uncased'

model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
tokenizer.tokenize("CHUPAME LA PIJA")

['chupa', '##me', 'la', 'pi', '##ja']

In [3]:
tokenizer.tokenize("pija")

['pi', '##ja']

In [4]:
tokenizer.tokenize("trolo")

['tro', '##lo']

In [5]:
tokenizer.tokenize("maricón")

['maricón']

In [6]:
tokenizer.tokenize("marica")

['marica']

In [7]:
tokenizer.tokenize("puto")

['puto']

In [8]:
tokenizer.tokenize("Hacete ortear viejo trolazo")

['hace', '##te', 'or', '##tear', 'viejo', 'tro', '##laz', '##o']

Es un problema esto. Veamos cómo agregar posiblemente estos tokens...

## Training new tokenizer

In [9]:
from hatedetection import load_datasets

train_dataset, dev_dataset, test_dataset = load_datasets()

In [10]:
new_tokenizer.train_from_iterator?

Object `new_tokenizer.train_from_iterator` not found.


In [11]:
BertWordPieceTokenizer??

Object `BertWordPieceTokenizer` not found.


In [12]:
from hatedetection.preprocessing import preprocess_tweet

preprocess_tweet("@clarincom jajajaja #NoVuelvenMas 🤣", hashtag_token="[HASHTAG]")

'[USER] jajajaja [HASHTAG] no vuelven mas 🤣'

Veamos los que están más de 10 veces

In [18]:
from tokenizers import BertWordPieceTokenizer

new_tokenizer = BertWordPieceTokenizer(lowercase=True)
texts = [ex["text"] for ex in train_dataset]
new_tokenizer.train_from_iterator(
    texts, min_frequency=50
)

In [19]:
new_tokenizer

Tokenizer(vocabulary_size=3408, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=True, wordpieces_prefix=##)

In [20]:
print(new_tokenizer.encode("Cría horrenda").tokens)

['cri', '##a', 'hor', '##ren', '##da']


In [21]:
old_tokens = set(tokenizer.get_vocab())

missing_tokens = [tok for tok in new_tokenizer.get_vocab() if tok not in old_tokens]

len(missing_tokens)

983

Hay 5217 tokens que no están! Es un montón...

In [22]:
i = 400

for i, tok in enumerate(sorted(missing_tokens)):
    print(f"{i+1:<4} -- {tok}")

1    -- #
2    -- ##aaa
3    -- ##acion
4    -- ##aj
5    -- ##aja
6    -- ##ajaj
7    -- ##ajaja
8    -- ##ajajaja
9    -- ##ajajajaj
10   -- ##ajajajajajajajaj
11   -- ##ajj
12   -- ##ajo
13   -- ##amb
14   -- ##ambien
15   -- ##arent
16   -- ##arentena
17   -- ##baj
18   -- ##bajo
19   -- ##ccion
20   -- ##chner
21   -- ##ciela
22   -- ##cras
23   -- ##cto
24   -- ##ctu
25   -- ##cub
26   -- ##cun
27   -- ##deces
28   -- ##demia
29   -- ##dena
30   -- ##dian
31   -- ##dical
32   -- ##dran
33   -- ##dres
34   -- ##dria
35   -- ##drian
36   -- ##entable
37   -- ##ereb
38   -- ##erica
39   -- ##esion
40   -- ##fiero
41   -- ##gles
42   -- ##greso
43   -- ##guenza
44   -- ##hhh
45   -- ##ican
46   -- ##ilita
47   -- ##inci
48   -- ##irus
49   -- ##isimo
50   -- ##ision
51   -- ##istina
52   -- ##izacion
53   -- ##lacion
54   -- ##licar
55   -- ##mbe
56   -- ##minar
57   -- ##mocra
58   -- ##nn
59   -- ##oba
60   -- ##obre
61   -- ##ocu
62   -- ##omb
63   -- ##ombo
64   -- ##onter
65   -

Hay que preprocesar esto, es una cagada así

Primero, emojis

In [84]:
import emoji

emoji.demojize("Viva Perón ✌✌✌✌✌✌✌✌✌✌✌")

'Viva Perón :victory_hand::victory_hand::victory_hand::victory_hand::victory_hand::victory_hand::victory_hand::victory_hand::victory_hand::victory_hand::victory_hand:'

In [21]:
import re
hashtag_regex = re.compile(r'\B#(\w*[a-zA-Z]+\w*)')

text = "Probando #PreprocesamientoDeHashtags a #versi anda bien #"
hashtag_regex.findall(text)

['PreprocesamientoDeHashtags', 'versi']

In [32]:
repeated_regex = re.compile(r"(.)(\1+)")

repeated_regex.findall("Holaaaa   tu viejaaaaaa")

[('a', 'aaa'), (' ', '  '), ('a', 'aaaaa')]

In [125]:
interspace = re.compile(r'([A-Z]+)')

interspace.sub(r' \1', "CamelCaseSarasa").lower().strip()

'camel case sarasa'

In [126]:

def sarasa(x):
    print(x)
    text = x.groups()[0]
    
    return camel_to_human(text)



hashtag_regex.sub(
    sarasa,
    text
)




<re.Match object; span=(9, 36), match='#PreprocesamientoDeHashtags'>
<re.Match object; span=(39, 45), match='#versi'>


'Probando Preprocesamiento De Hashtags a versi anda bien #'